In [1]:
# Update sklearn to prevent version mismatches
!pip install sklearn --upgrade

Requirement already up-to-date: sklearn in c:\users\anaro\anaconda3\lib\site-packages (0.0)


In [2]:
# install joblib. This will be used to save your model. 
# Restart your kernel after installing 
!pip install joblib

In [3]:
%matplotlib inline
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

In [4]:
from sklearn.ensemble import RandomForestClassifier

# Read the CSV and Perform Basic Data Cleaning

In [5]:
df = pd.read_csv("Resources/exoplanet_data.csv")
# Drop the null columns where all values are null
df = df.dropna(axis='columns', how='all')
# Drop the null rows
df = df.dropna()
df.head()

,koi_disposition,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,CONFIRMED,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,FALSE POSITIVE,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,FALSE POSITIVE,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,CONFIRMED,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,CONFIRMED,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


# Select your features (columns)

In [6]:
# Set features. This will also be used as your x values.
selected_features = df[['koi_fpflag_nt', 'koi_fpflag_ss', 'koi_fpflag_co',
       'koi_fpflag_ec', 'koi_period', 'koi_period_err1', 'koi_period_err2',
       'koi_time0bk', 'koi_time0bk_err1', 'koi_time0bk_err2', 'koi_impact',
       'koi_impact_err1', 'koi_impact_err2', 'koi_duration',
       'koi_duration_err1', 'koi_duration_err2', 'koi_depth', 'koi_depth_err1',
       'koi_depth_err2', 'koi_prad', 'koi_prad_err1', 'koi_prad_err2',
       'koi_teq', 'koi_insol', 'koi_insol_err1', 'koi_insol_err2',
       'koi_model_snr', 'koi_tce_plnt_num', 'koi_steff', 'koi_steff_err1',
       'koi_steff_err2', 'koi_slogg', 'koi_slogg_err1', 'koi_slogg_err2',
       'koi_srad', 'koi_srad_err1', 'koi_srad_err2', 'ra', 'dec',
       'koi_kepmag']]
selected_features.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
0,0,0,0,0,54.418383,2.479000e-04,-2.479000e-04,162.513840,0.003520,-0.003520,...,-81,4.467,0.064,-0.096,0.927,0.105,-0.061,291.93423,48.141651,15.347
1,0,1,0,0,19.899140,1.490000e-05,-1.490000e-05,175.850252,0.000581,-0.000581,...,-176,4.544,0.044,-0.176,0.868,0.233,-0.078,297.00482,48.134129,15.436
2,0,1,0,0,1.736952,2.630000e-07,-2.630000e-07,170.307565,0.000115,-0.000115,...,-174,4.564,0.053,-0.168,0.791,0.201,-0.067,285.53461,48.285210,15.597
3,0,0,0,0,2.525592,3.760000e-06,-3.760000e-06,171.595550,0.001130,-0.001130,...,-211,4.438,0.070,-0.210,1.046,0.334,-0.133,288.75488,48.226200,15.509
4,0,0,0,0,4.134435,1.050000e-05,-1.050000e-05,172.979370,0.001900,-0.001900,...,-232,4.486,0.054,-0.229,0.972,0.315,-0.105,296.28613,48.224670,15.714


In [7]:
# Assign the data to X and y
# Reshape
X = selected_features
y = df.koi_disposition.values.reshape(-1, 1)
print("Shape: ", X.shape, y.shape)
X, y

Shape:  (6991, 40) (6991, 1)


(      koi_fpflag_nt  koi_fpflag_ss  koi_fpflag_co  koi_fpflag_ec  koi_period  \
 0                 0              0              0              0   54.418383   
 1                 0              1              0              0   19.899140   
 2                 0              1              0              0    1.736952   
 3                 0              0              0              0    2.525592   
 4                 0              0              0              0    4.134435   
 ...             ...            ...            ...            ...         ...   
 6986              0              0              0              1    8.589871   
 6987              0              1              1              0    0.527699   
 6988              0              0              0              0    1.739849   
 6989              0              0              1              0    0.681402   
 6990              0              0              1              1    4.856035   
 
       koi_period_err1  ko

# Create a Train Test Split

Use `koi_disposition` for the y values

In [8]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

In [9]:
X_train.head()

,koi_fpflag_nt,koi_fpflag_ss,koi_fpflag_co,koi_fpflag_ec,koi_period,koi_period_err1,koi_period_err2,koi_time0bk,koi_time0bk_err1,koi_time0bk_err2,...,koi_steff_err2,koi_slogg,koi_slogg_err1,koi_slogg_err2,koi_srad,koi_srad_err1,koi_srad_err2,ra,dec,koi_kepmag
3563,0,0,0,0,10.548413,5.470000e-05,-5.470000e-05,139.064020,0.004110,-0.004110,...,-133,4.387,0.066,-0.123,1.092,0.181,-0.097,298.09543,44.737061,13.204
4099,0,0,0,0,24.754385,1.365000e-04,-1.365000e-04,140.207320,0.004460,-0.004460,...,-144,4.519,0.078,-0.052,0.804,0.056,-0.076,295.73535,42.576248,15.514
5460,0,0,0,0,1.057336,1.230000e-07,-1.230000e-07,131.792007,0.000096,-0.000096,...,-140,4.594,0.054,-0.027,0.683,0.054,-0.060,292.18417,49.310040,15.414
1091,0,0,0,0,201.118319,1.461000e-03,-1.461000e-03,187.569860,0.005290,-0.005290,...,-112,4.447,0.072,-0.108,0.954,0.135,-0.083,283.11377,48.131390,13.328
5999,0,0,0,0,91.649983,3.181000e-03,-3.181000e-03,175.715600,0.028600,-0.028600,...,-233,4.145,0.164,-0.164,1.608,0.905,-0.383,294.93198,39.812420,12.964


In [10]:
y_train

array([['CANDIDATE'],
       ['CONFIRMED'],
       ['CANDIDATE'],
       ...,
       ['FALSE POSITIVE'],
       ['CONFIRMED'],
       ['CONFIRMED']], dtype=object)

# Pre-processing

Scale the data using the MinMaxScaler and perform some feature selection

In [12]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_train, y_train)

<ipython-input-12-a8455e56379c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = rf.fit(X_train, y_train)


1.0

In [13]:
rf.get_params()

{'bootstrap': True,
 'ccp_alpha': 0.0,
 'class_weight': None,
 'criterion': 'gini',
 'max_depth': None,
 'max_features': 'auto',
 'max_leaf_nodes': None,
 'max_samples': None,
 'min_impurity_decrease': 0.0,
 'min_impurity_split': None,
 'min_samples_leaf': 1,
 'min_samples_split': 2,
 'min_weight_fraction_leaf': 0.0,
 'n_estimators': 200,
 'n_jobs': None,
 'oob_score': False,
 'random_state': None,
 'verbose': 0,
 'warm_start': False}

In [14]:
importances = rf.feature_importances_
importances

array([0.09335763, 0.07057246, 0.10238573, 0.03656483, 0.02104783,
       0.01697714, 0.01664113, 0.01466685, 0.02192659, 0.02462229,
       0.018744  , 0.01099911, 0.01118702, 0.02317641, 0.0343635 ,
       0.03097938, 0.01969179, 0.01445702, 0.01404511, 0.04695711,
       0.0324587 , 0.03288604, 0.01580505, 0.01583706, 0.02121211,
       0.01621517, 0.05520268, 0.00318996, 0.00972776, 0.03185296,
       0.02887015, 0.00941814, 0.0092219 , 0.01019376, 0.00895972,
       0.01186836, 0.0086214 , 0.01278495, 0.01131179, 0.01099941])

In [15]:
pd.DataFrame(sorted(zip(rf.feature_importances_, X_train.columns), reverse=True))

,0,1
0,0.102386,koi_fpflag_co
1,0.093358,koi_fpflag_nt
2,0.070572,koi_fpflag_ss
3,0.055203,koi_model_snr
4,0.046957,koi_prad
5,0.036565,koi_fpflag_ec
6,0.034364,koi_duration_err1
7,0.032886,koi_prad_err2
8,0.032459,koi_prad_err1
9,0.031853,koi_steff_err1


In [31]:
rf.predict_proba(X_test)
rf.classes_

array(['CANDIDATE', 'CONFIRMED', 'FALSE POSITIVE'], dtype=object)

In [32]:
# Set features. This will also be used as your x values.
best_features = df[['koi_fpflag_co', 'koi_fpflag_nt', 'koi_fpflag_ss', 'koi_model_snr',
                    'koi_prad', 'koi_duration_err1', 'koi_fpflag_ec', 'koi_prad_err1',
                    'koi_prad_err2', 'koi_duration_err2', 'koi_steff_err1']]
best_features.head()

,koi_fpflag_co,koi_fpflag_nt,koi_fpflag_ss,koi_model_snr,koi_prad,koi_duration_err1,koi_fpflag_ec,koi_prad_err1,koi_prad_err2,koi_duration_err2,koi_steff_err1
0,0,0,0,25.8,2.83,0.11600,0,0.32,-0.19,-0.11600,81
1,0,0,1,76.3,14.60,0.03410,0,3.92,-1.31,-0.03410,158
2,0,0,1,505.6,33.46,0.00537,0,8.50,-2.83,-0.00537,157
3,0,0,0,40.9,2.75,0.04200,0,0.88,-0.35,-0.04200,169
4,0,0,0,40.2,2.77,0.06730,0,0.90,-0.30,-0.06730,189


In [33]:
# Assign the data to X and y
# Reshape
X = best_features
y = df.koi_disposition.values.reshape(-1, 1)
print("Shape: ", X.shape, y.shape)
X, y

Shape:  (6991, 11) (6991, 1)


(      koi_fpflag_co  koi_fpflag_nt  koi_fpflag_ss  koi_model_snr  koi_prad  \
 0                 0              0              0           25.8      2.83   
 1                 0              0              1           76.3     14.60   
 2                 0              0              1          505.6     33.46   
 3                 0              0              0           40.9      2.75   
 4                 0              0              0           40.2      2.77   
 ...             ...            ...            ...            ...       ...   
 6986              0              0              0            8.4      1.11   
 6987              1              0              1          453.3     29.35   
 6988              0              0              0           10.6      0.72   
 6989              1              0              0           12.3      1.07   
 6990              1              0              0            8.2      1.05   
 
       koi_duration_err1  koi_fpflag_ec  koi_prad_

In [34]:
from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=1)

# Train the Model



In [26]:
# Scale your data
# from sklearn.preprocessing import MinMaxScaler
# X_minmax = MinMaxScaler().fit(X_train)

# X_train_minmax = X_minmax.transform(X_train)
# X_test_minmax = X_minmax.transform(X_test)

In [27]:
# from sklearn import preprocessing
# from sklearn.preprocessing import LabelEncoder

# label_encoder = LabelEncoder()
# label_encoder.fit(y_train)
# encoded_y_train = label_encoder.transform(y_train)
# encoded_y_test = label_encoder.transform(y_test)
# encoded_y_train

In [28]:
# from sklearn.linear_model import LogisticRegression
# classifier = LogisticRegression(C=1, intercept_scaling=0.01)
# classifier.get_params()

In [29]:
# classifier.fit(X_train, encoded_y_train)

In [30]:
rf = RandomForestClassifier(n_estimators=200)
rf = rf.fit(X_train, y_train)
rf.score(X_train, y_train)

<ipython-input-30-a8455e56379c>:2: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples,), for example using ravel().
  rf = rf.fit(X_train, y_train)


1.0

In [119]:
# Calculate the mean_squared_error and the r-squared value
from sklearn.metrics import mean_squared_error, r2_score

predicted = classifier.predict(X_test)

mse = mean_squared_error(encoded_y_test, predicted)
r2 = r2_score(encoded_y_test, predicted)

print('mse: ', mse, 'r2: ', r2)

mse:  0.14473684210526316 r2:  0.7832242454900473


In [120]:
print(f"Training Data Score: {classifier.score(X_train, encoded_y_train)}")
print(f"Testing Data Score: {classifier.score(X_test, encoded_y_test)}")

Training Data Score: 0.8460804882700744
Testing Data Score: 0.8655606407322655


# Hyperparameter Tuning

Use `GridSearchCV` to tune the model's parameters

In [121]:
# Create the GridSearchCV model
from sklearn.model_selection import GridSearchCV
param_grid = {'C': [1, 5, 10],
              'intercept_scaling':[0.01, 0.1, 1]}
grid = GridSearchCV(classifier, param_grid, verbose=3)

In [122]:
classifier.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'l1_ratio', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [123]:
grid.fit(X_train, encoded_y_train)

Fitting 5 folds for each of 9 candidates, totalling 45 fits
[CV] C=1, intercept_scaling=0.01 .....................................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
C:\Users\anaro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] ......... C=1, intercept_scaling=0.01, score=0.837, total=   0.2s
[CV] C=1, intercept_scaling=0.01 .....................................


C:\Users\anaro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done   2 out of   2 | elapsed:    0.4s remaining:    0.0s
C:\Users\anaro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https:/

[CV] ......... C=1, intercept_scaling=0.01, score=0.805, total=   0.3s
[CV] C=1, intercept_scaling=0.01 .....................................
[CV] ......... C=1, intercept_scaling=0.01, score=0.761, total=   0.2s
[CV] C=1, intercept_scaling=0.01 .....................................


...

[CV] ........... C=10, intercept_scaling=1, score=0.821, total=   0.3s
[CV] C=10, intercept_scaling=1 .......................................


C:\Users\anaro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
[Parallel(n_jobs=1)]: Done  45 out of  45 | elapsed:   10.0s finished


[CV] ........... C=10, intercept_scaling=1, score=0.805, total=   0.3s


C:\Users\anaro\anaconda3\lib\site-packages\sklearn\linear_model\_logistic.py:762: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(


GridSearchCV(estimator=LogisticRegression(C=1, intercept_scaling=0.01),
             param_grid={'C': [1, 5, 10], 'intercept_scaling': [0.01, 0.1, 1]},
             verbose=3)

In [124]:
print(grid.best_params_)

{'C': 10, 'intercept_scaling': 0.01}


In [125]:
predictions = grid.predict(X_test)

In [126]:
print('Test Acc: %.3f' % grid.score(X_test, encoded_y_test))

Test Acc: 0.851


In [127]:
# Train the model with GridSearch
from sklearn.metrics import classification_report
print(classification_report(encoded_y_test, predictions,
                            target_names=['CANDIDATE', 'FALSE POSITIVE', 'CONFIRMED']))

                precision    recall  f1-score   support

     CANDIDATE       0.77      0.56      0.65       404
FALSE POSITIVE       0.67      0.83      0.74       435
     CONFIRMED       0.98      0.99      0.99       909

      accuracy                           0.85      1748
     macro avg       0.81      0.79      0.79      1748
  weighted avg       0.86      0.85      0.85      1748



In [128]:
print(grid.best_params_)
print(grid.best_score_)

{'C': 10, 'intercept_scaling': 0.01}
0.814990649036887


# Save the Model

In [129]:
# save your model by updating "your_name" with your name
# and "your_model" with your model variable
# be sure to turn this in to BCS
# if joblib fails to import, try running the command to install in terminal/git-bash
import joblib
filename = 'Craig_Matherson_Model_2.sav'
joblib.dump(classifier, filename)

['Craig_Matherson_Model_2.sav']